<a href="https://colab.research.google.com/github/uirom/practice/blob/main/%EC%A0%9C_2%EC%9C%A0%ED%98%95_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%5B2_%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5_%EB%B6%84%EB%A5%98%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**의사결정나무**

In [ ]:
# 의사결정나무를 이용 - 타이타닉 생존자 분류(예측) 수행

import numpy as np
import pandas as pd
import sklearn

# 의사결정나무 분류모델을 위한 패키지 임포트
from sklearn.tree import DecisionTreeClassifier
# 학습 및 테스트 데이터셋 분리를 위한 패키지 임포트
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/titanic.csv")

In [ ]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 데이터 값의 결측값 존재하는지 확인 - df.info()사용
df.info()

# 891이 아닌 null값들은 모두 결측값 존재함을 확인 - Age, Cabin, Embarked

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# 데이터프레임의 describe()이용해 데이터의 요약정보도 확인
df.describe()

# describe 이용 이유: 전체 데이터의 개수, 각 컬럼별 평균, 표준편차, 최대값, 최소값 등 데이터 분포를 확인

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# 데이터 전처리1 - (전처리 해야하는 함수: Age, Cabin, Embarked)
# Cabin - 결측치가 너무 많기 때문에 분석에서 제외

# Age - 평균값으로 결측값 대치
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

# Embarked - 최빈값으로 결측값 대치
d_mode = df["Embarked"].mode()
df["Embarked"].fillna(d_mode, inplace=True)


In [ ]:
# 데이터 전처리2 - Sex 컬럼은 이전의 텍스트값 -> 숫자 0과 1로 변환하는 레이블 인코딩 수행
#               Embarked 컬럼도 텍스트값을 숫자로 변환하는 레이블 인코딩 수행  => 사이킷런의 LabelEncoder()사용

# Sex 컬럼의 값을 1과 0으로 레이블 인코딩 한다.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder
df["Sex"] = le().fit_transform(df["Sex"])

# Embarked 컬럼의 값을 레이블 인코딩 한다.
from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

In [ ]:
# 데이터 전처리3 - 동승가족수는 SibSP(형제 또는 자매) + Parch(부모 또는 자녀수) = FamilySize(가족) 로 파생변수 생성

df["FamilySize"] = df["SibSp"] + df["Parch"]

In [ ]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,2
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,2
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,0


In [ ]:
# 분석 데이터셋 준비 -> 전처리를 마친 데이터를 학습용 데이터셋과 테스트 데이터셋으로 분리 (8:2)
# if) 학습용 데이터셋과 테스트용 데이터셋이 각각 주어지면 이 과정 생략 가능

X = df[["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]]
y = df["Survived"]

In [ ]:
X

,Pclass,Sex,Age,Fare,Embarked,FamilySize
0,3,1,22.000000,7.2500,2,1
1,1,0,38.000000,71.2833,0,1
2,3,0,26.000000,7.9250,2,0
3,1,0,35.000000,53.1000,2,1
4,3,1,35.000000,8.0500,2,0
...,...,...,...,...,...,...
886,2,1,27.000000,13.0000,2,0
887,1,0,19.000000,30.0000,2,0
888,3,0,29.699118,23.4500,2,3
889,1,1,26.000000,30.0000,0,0


In [ ]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
# 분석 데이터셋 분할 (8:2) - test_size=0.2는 테스트 데이터 20%로 분리함을 뜻함.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
# shape 속성을 이용해 결과 데이터셋의 크기 살펴보기

print(X_train.shape)    # 학습 데이터(독립변수)
print(X_test.shape)     # 테스트 데이터(독립변수)
print(y_train.shape)    # 학습 데이터(종속변수)
print(y_test.shape)     # 테스트 데이터(종속변수)

# 결과 확인: 학습 데이터셋은 712개, 테스트 데이터는 179개로 분할됨을 확인

(712, 6)
(179, 6)
(712,)
(179,)


In [ ]:
# 데이터 분석 수행1 - DecisionTreeClassifier 객체 생성
dt = DecisionTreeClassifier(random_state=11)
dt.fit(X_train, y_train)   # 학습 수행

DecisionTreeClassifier(random_state=11)

In [ ]:
# 데이터 분석 수행2 - 학습이 완료된 dt객체에서 테스트 데이터셋으로 분류(예측) 생성
# 분류(예측) 수행은 predict()함수에 테스트 데이터셋 X_test를 입력값으로 주고, 분류(예측) 수행 후 결과를 변수(pred)에 저장

pred = dt.predict(X_test)

In [ ]:
# 성능평가 및 시각화 - sklearn의 accracy_score()로 정확도 측정

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 결과 확인: 모델 정확도 78%

0.7877094972067039


In [ ]:
# 타이타닉 생존자 분류(예측)하는 전체 코드

import numpy as np
import pandas as pd
import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/titanic.csv")

d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

d_mode = df["Embarked"].mode()
df["Embarked"].fillna(d_mode, inplace=True)

from sklearn.preprocessing import LabelEncoder
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

df["FamilySize"] = df["SibSp"] + df["Parch"]

X = df[["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]]
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

dt = DecisionTreeClassifier(random_state=11)

dt.fit(X_train, y_train)

pred = dt.predict(X_test)

from sklearn.metrics import accuracy_score
acc= accuracy_score(y_test, pred)
print(acc)

0.7877094972067039


KNN **알고리즘**

In [ ]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/iris.csv")

In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
df.info()

# 모두 150. num-null으로 결측값 없음을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
# 데이터프레임의 describe()사용하여 데이터의 요약정보 확인

df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
# 데이터 전처리 - 4개 독립변수에 대해 Min-Max 정규화 (모든 값을 0~1사이의 값으로 변환)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])

In [ ]:
# 분석 데이터셋 준비 - 데이터 분리를 위한 사이킷런의 train_test_split()함수 사용
# X는 독립변수(설명변수), y는 종속변수(목표변수)

X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
# shape 속성으로 값 확인
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# 학습데이터셋은 120, 테스트데이터셋은 30으로 나눠짐을 확인

(120, 4)
(30, 4)
(120,)
(30,)


In [ ]:
# 데이터 분석 수행1 - 분류를 위한 알고리즘 중에서 KNN 이용

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
# 데이터 분석 수행2 - 학습이 완료된 knn객체에서 테스트 데이터셋으로 분류(예측)수행

pred = knn.predict(X_test)

In [ ]:
# 성능평가 및 시각화
# 분류(예측) 결과(pred)와 실제 분류 결과(y_test)를 비교하여 정확도 평가

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 93%의 정확도로 붓꽃종류 예측 가능

0.9333333333333333


In [ ]:
# 전체 코드 - n_neighbors의 값을 3, 6, 9로 변경하면서 정확도 값을 비교해본다

import numpy as np
import pandas as pd
import sklearn

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/iris.csv")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])

X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 93%

0.9333333333333333


응용: KNN을 의사결정나무의 코드를 변경해서 사용해보기

In [ ]:
# 의사결정나무를 이용한 분류문제 해결에 사용했던 코드 KNN으로 변경 해서 사용해보기
# KNeighborsClassifier 객체 생성

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)   # 학습 수행

# 학습이 완료된 dt객체에서 테스트 데이터셋으로 예측 수행
pred = knn.predict(X_test)

In [ ]:
# sklearn의 오차행렬[confusion_matrix()] 이용

from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

[[ 9  0  0]
 [ 0 10  0]
 [ 0  2  9]]


In [ ]:
# matrics 모듈의 classification_report() 함수 이용한 평가지표 계산

from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       0.83      1.00      0.91        10
   virginica       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30



SVM **이용**

In [ ]:
# ①: 필요 패키지 임포트

import numpy as np
import pandas as pd
import sklearn

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
# ②: 데이터 불러오기

df = pd.read_csv("/content/titanic.csv")

In [ ]:
# ③: 데이터 살펴보기
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df.info()

# 결측치: Age, Embarked, Cabin

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# ④ : 데이터 전처리 수행
# Age= 결측치-> 평균값으로 대체, Embarked-> 최빈값으로 대체, Cabin 결측치 너무 많아 분석에서 제외

# Age : 결측값 => 평균으로 대체
d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

# Embarked : 결측값 => 최빈값으로 대체
d_mode = df["Embarked"].mode()[0]    # mode()[0]하는 이유는 mode()함수는 항상 시리즈 상관없이 값을 반환하기 때문
df["Embarked"].fillna(d_mode, inplace=True)

# SibSp + Parch = FamilySize 파생변수 생성하기
df["FamilySize"] = df["SibSp"] + df["Parch"]

In [ ]:
# ④ : 데이터 전처리 수행2
# Sex: 텍스트 => 숫자 0, 1로 변환하는 원핫 인코딩 사용
onehot_sex = pd.get_dummies(df["Sex"])
df = pd.concat([df, onehot_sex], axis=1)

# Embarked: 텍스트 => 숫자 0, 1로 변환하는 원핫 인코딩 사용
onehot_Embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, onehot_Embarked], axis=1)

In [ ]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,female,male,C,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,1,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,1,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,1,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,0,0,1,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,0,1,0,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,3,1,0,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,0,0,1,1,0,0


In [ ]:
# 전처리 끝나고 분석 데이터셋 준비

# 목표변수(종속변수) : Survived
# 독립변수 : Pcalss, Age, Fare, FamilySize, female, male, C, Q, S

X = df[["Pclass", "Age", "Fare", "FamilySize", "female", "male", "C", "Q", "S"]]
y = df["Survived"]

In [ ]:
# 데이터 분리 (데이터셋 분할 =7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
# train은 학습 데이터, test는 테스트 데이터
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# 학습 데이터셋 623개, 테스트 데이터 268개로 분할됨을 확인

(623, 9)
(268, 9)
(623,)
(268,)


In [ ]:
from sklearn import svm
# 데이터 분석 수행. 주어진 데이터로 탑승자의 생존을 구분하는 문제는 분류문제임
# 분류를 위한 알고리즘 중에서 SVM을 이용

sv = svm.SVC(kernel='rbf')
sv.fit(X_train, y_train)

# 저차원의 공간을 고차원의 공간으로 매핑해주는 작업(커널 트릭)을 함으로써 경계면을 구한다.

SVC()

In [ ]:
# 분류(예측)은 predict()함수에 테스트 데이터셋 X_test를 입력값으로 주고, X_test에 대해 분류 수행되며 결과를 지정한 변수(pred)에 지정
# 학습이 완료된 dt객체에서 테스트 데이터셋으로 예측 수행
pred = sv.predict(X_test)

In [ ]:
# 성능평가 - sklearn의 accuracy_score을 이용
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 정확도 72%

0.7238805970149254


In [ ]:
# 성능평가 - matrics 모듈의 confusion_matrix()함수 사용하여 오차행렬 구하기

from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

# 결과 확인: 정확하게 분류한 데이터는 194(167+27)개이며, 잘못 분류한 데이터는 74(67+7)개이다.
#          TP: 167, TN: 55, FP:7, FN:67

[[167   7]
 [ 67  27]]


In [ ]:
# 성능평가 - matrics 모듈의 classification_report()함수를 이용한 평가지표 계산

from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

# 결과 확인: fl-score=미생존자(0) 예측의 정확도는 0.82, 생존자(1)의 예측의 정확도는 0.42로 차이가 있음을 확인.

              precision    recall  f1-score   support

           0       0.71      0.96      0.82       174
           1       0.79      0.29      0.42        94

    accuracy                           0.72       268
   macro avg       0.75      0.62      0.62       268
weighted avg       0.74      0.72      0.68       268



In [ ]:
# 전체 코드

import numpy as np
import pandas as pd
import sklearn

from sklearn import svm
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/titanic.csv")

d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)

d_mode = df["Embarked"].mode()
df["Embarked"].fillna(d_mode, inplace=True)

df["FamilySize"] = df["SibSp"] + df["Parch"]

onehot_sex = pd.get_dummies(df["Sex"])
df = pd.concat([df, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(df["Embarked"])
df = pd.concat([df, onehot_embarked], axis=1)

X = df[["Pclass", "Age", "Fare", "FamilySize", "female", "male", "C", "Q", "S"]]
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

sv=svm.SVC(kernel='rbf')
sv.fit(X_train, y_train)

pred=sv.predict(X_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

0.7238805970149254
[[167   7]
 [ 67  27]]
              precision    recall  f1-score   support

           0       0.71      0.96      0.82       174
           1       0.79      0.29      0.42        94

    accuracy                           0.72       268
   macro avg       0.75      0.62      0.62       268
weighted avg       0.74      0.72      0.68       268



In [ ]:
# SVM 커널 파라미터 조정
# SVM - C(비용), gamma(허용 표준편차) 이용해서 결정경계 조정 가능하다. gamma가 커지면 표준편차가 작아지고, 결정경계도 작아지면서 구부러진다.

# 커널 파라미터 이용 - rbf 적용
sv = svm.SVC(kernel = 'rbf')

# 커널 파라미터- linear 적용, C=1, gamma=0.1
sv = svm.SVC(kernel='linear', C=1, gamma=0.1)

# 커널 파라미터 - rbf 적용, C=0.1, gamma=0.1
sv = svm.SVC(kernel='rbf', C=1, gamma=0.1)

로지스틱 **회귀** (회귀지만 분류인거 잊지말기)

In [ ]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/iris.csv")

In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
# 데이터 전처리 - 4개의 독립변수에 대해 Min-Max 정규화 시키기

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])

In [ ]:
# 분석 데이터셋 준비

X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = df["species"]

# 분석 데이터셋 분할 (8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
# shape 속성을 이용하여, 결과 데이터셋의 크기 살펴보기

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120,)
(30,)


In [ ]:
# 분류를 위한 알고리즘 중에서 로지스틱 회귀를 이용

lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# 학습이 완료된 lr객체에서 테스트 데이터셋으로 분류(예측) 수행
pred = lr.predict(X_test)

In [ ]:
# 성능평가 및 시각화
# 분류(예측) 결과(pred)와 실제 분류 결과(Y_test)를 비교하여 정확도 평가

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.8333333333333334


In [ ]:
# 전체 코드

랜덤 **포레스트**

In [ ]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/titanic.csv")

In [ ]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# 데이터 전처리

d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

In [ ]:
# 데이터 전처리 - 레이블 인코딩

from sklearn.preprocessing import LabelEncoder
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

In [ ]:
df["FamilySize"] = df["SibSp"] + df["Parch"]

In [ ]:
# 분석 데이터셋 준비. 전처리를 마친 데이터를 학습용 데이터셋과 테스트용 데이터셋으로 분리

X = df[["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]]
y = df["Survived"]

In [ ]:
X

,Pclass,Sex,Age,Fare,Embarked,FamilySize
0,3,1,22.000000,7.2500,2,1
1,1,0,38.000000,71.2833,0,1
2,3,0,26.000000,7.9250,2,0
3,1,0,35.000000,53.1000,2,1
4,3,1,35.000000,8.0500,2,0
...,...,...,...,...,...,...
886,2,1,27.000000,13.0000,2,0
887,1,0,19.000000,30.0000,2,0
888,3,0,29.699118,23.4500,2,3
889,1,1,26.000000,30.0000,0,0


In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# 학습데이터: 712개, 테스트 데이터: 179개

(712, 6)
(179, 6)
(712,)
(179,)


In [ ]:
# 데이터 분석 수행 - 분류를 위한 알고리즘 중에서 랜덤 포레스트 이용

rf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=20)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, n_estimators=50, random_state=20)

In [ ]:
# 학습이 완료된 rf 객체에서 테스트데이터셋으로 분류(예측) 수행
pred = rf.predict(X_test)

In [ ]:
# 성능 평가
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 86% 정도의 성능을 보인다

0.8603351955307262


In [ ]:
# 전체 코딩
import numpy as np
import pandas as pd
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/titanic.csv")

d_mean = df["Age"].mean()
df["Age"].fillna(d_mean, inplace=True)
d_mode = df["Embarked"].mode()[0]
df["Embarked"].fillna(d_mode, inplace=True)

from sklearn.preprocessing import LabelEncoder
df["Sex"] = LabelEncoder().fit_transform(df["Sex"])

from sklearn.preprocessing import LabelEncoder
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

df["FamilySize"] = df["SibSp"] + df["Parch"]

X = df[["Pclass", "Age", "Sex", "Fare", "Embarked", "Fare", "FamilySize"]]
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

rf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=20)
rf.fit(X_train, y_train)

pred = rf.predict(X_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

# 전처리때문에 결과가 다를 수 있음

0.8435754189944135
